In [1]:
import pandas as pd
from difflib import SequenceMatcher
import re
from collections import defaultdict

def normalize_string(s):
    """Normalize string by removing special characters and extra spaces"""
    return re.sub(r'[^\w\s]', '', s.lower()).strip()

def string_similarity(a, b):
    """Calculate similarity ratio between two strings"""
    return SequenceMatcher(None, normalize_string(a), normalize_string(b)).ratio()

def find_similar_groups(df):
    """Find groups of similar store names in the dataset"""
    unique_stores = df['КонкурентМагазин'].unique()
    n = len(unique_stores)
    
    # Create graph of similar names
    similarity_graph = defaultdict(set)
    
    # Compare each pair
    for i in range(n):
        for j in range(i + 1, n):
            store1 = unique_stores[i]
            store2 = unique_stores[j]
            
            if string_similarity(store1, store2) > 0.85:
                similarity_graph[store1].add(store2)
                similarity_graph[store2].add(store1)
    
    # Find connected components (groups of similar names)
    similar_groups = []
    visited = set()
    
    def dfs(store, current_group):
        """Depth-first search to find all connected similar names"""
        visited.add(store)
        current_group.add(store)
        for neighbor in similarity_graph[store]:
            if neighbor not in visited:
                dfs(neighbor, current_group)
    
    # Find all groups
    for store in unique_stores:
        if store not in visited:
            current_group = set()
            dfs(store, current_group)
            if len(current_group) > 1:  # Only include groups with multiple stores
                similar_groups.append(current_group)
    
    # Create detailed groups with links
    detailed_groups = []
    for group in similar_groups:
        group_details = {
            'stores': list(group),
            'links': {}
        }
        # Get links for each store in group
        for store in group:
            group_details['links'][store] = df[df['КонкурентМагазин'] == store]['Ссылки'].tolist()
        detailed_groups.append(group_details)
    
    return detailed_groups

# Read Excel file
file_path = r'C:\Users\Stukova_a\Documents\Сселеры МП.xlsx'
try:
    df = pd.read_excel(file_path)
    print("Первые несколько строк файла:")
    print(df.head())
    print("\nКолонки в файле:")
    print(df.columns.tolist())
    
    # Find similar name groups
    similar_groups = find_similar_groups(df)
    
    # Prepare detailed analysis text and store data
    detailed_analysis = ["Найдены группы похожих названий магазинов:\n" + "-" * 80 + "\n"]
    output_rows = []
    
    # Process each group
    for i, group in enumerate(similar_groups, 1):
        # Add group analysis text
        group_analysis = f"\nГруппа {i}:\n{'-' * 40}\n"
        
        # Add details for each store in group
        for store in group['stores']:
            group_analysis += f"Оригинальное название: '{store}'\n"
            group_analysis += f"Нормализованное название: '{normalize_string(store)}'\n"
            
        # Add similarity matrix for the group
        group_analysis += "\nМатрица схожести:\n"
        stores = group['stores']
        for store1 in stores:
            for store2 in stores:
                if store1 < store2:  # Only show each pair once
                    similarity = string_similarity(store1, store2)
                    group_analysis += f"'{store1}' <-> '{store2}': {similarity:.2f}\n"
        
        group_analysis += f"{'-' * 40}\n"
        detailed_analysis.append(group_analysis)
        
        # Add store-link pairs to output
        for store in group['stores']:
            for link in group['links'][store]:
                output_rows.append([store, link])
    
    # Create DataFrame and sort in reverse order
    output_df = pd.DataFrame(output_rows, columns=['КонкурентМагазин', 'Ссылки'])
    output_df = output_df.sort_index(ascending=False)
    
    # Save detailed analysis to text file
    analysis_path = r'C:\Users\Stukova_a\Documents\similar_stores_analysis2.txt'
    with open(analysis_path, 'w', encoding='utf-8') as f:
        f.writelines(detailed_analysis)
    print(f"\nПодробный анализ сохранен в файл: {analysis_path}")
    
    # Save store-links data to CSV
    output_path = r'C:\Users\Stukova_a\Documents\similar_stores_output2.csv'
    output_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Данные магазинов со ссылками сохранены в файл: {output_path}")
    
    # Print results to console
    print("\n".join(detailed_analysis))

except Exception as e:
    print(f"Ошибка при обработке файла: {str(e)}")

Первые несколько строк файла:
  КонкурентМагазин                                             Ссылки
0    ! Южный Берег  https://www.ozon.ru/product/aromaticheskoe-par...
1    ! Южный Берег  https://www.ozon.ru/product/krymskie-masla-aro...
2    ! Южный Берег  https://www.ozon.ru/product/aromaticheskie-par...
3    ! Южный Берег  https://www.ozon.ru/product/krymskie-masla-aro...
4    ! Южный Берег  https://www.ozon.ru/product/krymskie-masla-aro...

Колонки в файле:
['КонкурентМагазин', 'Ссылки']

Подробный анализ сохранен в файл: C:\Users\Stukova_a\Documents\similar_stores_analysis2.txt


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,КонкурентМагазин,Ссылки
359610,Яркий фотомаркет Юг,https://www.ozon.ru/product/batareyki-opticell...
359609,Яркий фотомаркет СЗ,https://www.ozon.ru/product/batareyki-opticell...
359608,Японские товары для здоровья и долголетия.,https://www.wildberries.ru/catalog/188151689/d...
359607,Японские товары для здоровья и долголетия.,https://www.wildberries.ru/catalog/179105021/d...
359606,Японские товары для здоровья и долголетия.,https://www.wildberries.ru/catalog/179732597/d...
...,...,...
4,ТероПром,https://www.ozon.ru/product/vatnye-palochki-em...
3,ТеропромЫЧ,https://www.ozon.ru/product/ultratonkie-dyshas...
2,ТеропромЫЧ,https://www.ozon.ru/product/krem-dlya-ruk-zd-g...
1,ТеропромЫЧ,https://www.ozon.ru/product/krem-dlya-ruk-smya...
